# ONS Trade in goods: country-by-commodity, imports and exports

This data is split into two distributions, one for imports and the other for exports:

https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/datasets/uktradecountrybycommodityimports

and

https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/datasets/uktradecountrybycommodityexports   

In [1]:
from gssutils import *

In [3]:
def run_script(s):
    %run "$s"
    return table

observations = pd.concat(
    run_script(s) for s in ['exports.ipynb', 'imports.ipynb']
).drop_duplicates()

Index(['0 Food & live animals', '00 Live animals',
       '01 Meat & meat preparations', '02 Dairy products & eggs',
       '03 Fish & shellfish', '04 Cereals', '05 Vegetables & fruit',
       '06 Sugar', '07 Coffee, tea, cocoa etc', '08 Animal feeding stuffs',
       ...
       '897C Jewellery', '89I Other manufactures (intermediate)',
       '89K Other manufactures (capital)',
       '89OC Other manufactures (consumer)', '8O Other manufactures',
       '8OC Other miscellaneous manufactures (consumer)',
       '8OI Other miscellaneous manufactures (intermediate)',
       '8OK Other miscellaneous manufactures (capital)', '9 Unspecified goods',
       'T Total'],
      dtype='object', length=125)

Index(['AD Andorra', 'AE United Arab Emirates', 'AF Afghanistan',
       'AG Antigua & Barbuda', 'AI Anguilla', 'AL Albania', 'AM Armenia',
       'AO Angola', 'AQ Antarctica', 'AR Argentina',
       ...
       'VN Vietnam', 'VU Vanuatu', 'W1 Whole world', 'WF Wallis & Futuna',
       'WS Samoa', 'XK Kosovo', 'YE Yemen', 'ZA South Africa', 'ZM Zambia',
       'ZW Zimbabwe'],
      dtype='object', length=237)

Index(['0', '00', '01', '02', '03', '04', '05', '06', '07', '08',
       ...
       '897C', '89I', '89K', '89OC', '8O', '8OC', '8OI', '8OK', '9', 'T'],
      dtype='object', length=125)

Index(['AD', 'AE', 'AF', 'AG', 'AI', 'AL', 'AM', 'AO', 'AQ', 'AR',
       ...
       'VN', 'VU', 'W1', 'WF', 'WS', 'XK', 'YE', 'ZA', 'ZM', 'ZW'],
      dtype='object', length=237)

Index(['0 Food & live animals', '00 Live animals',
       '01 Meat & meat preparations', '02 Dairy products & eggs',
       '03 Fish & shellfish', '04 Cereals', '05 Vegetables & fruit',
       '06 Sugar', '07 Coffee, tea, cocoa etc', '08 Animal feeding stuffs',
       ...
       '897C Jewellery', '89I Other manufactures (intermediate)',
       '89K Other manufactures (capital)',
       '89OC Other manufactures (consumer)', '8O Other manufactures',
       '8OC Other miscellaneous manufactures (consumer)',
       '8OI Other miscellaneous manufactures (intermediate)',
       '8OK Other miscellaneous manufactures (capital)', '9 Unspecified goods',
       'T Total'],
      dtype='object', length=125)

Index(['AD Andorra', 'AE United Arab Emirates', 'AF Afghanistan',
       'AG Antigua & Barbuda', 'AI Anguilla', 'AL Albania', 'AM Armenia',
       'AO Angola', 'AQ Antarctica', 'AR Argentina',
       ...
       'VN Vietnam', 'VU Vanuatu', 'W1 Whole world', 'WF Wallis & Futuna',
       'WS Samoa', 'XK Kosovo', 'YE Yemen', 'ZA South Africa', 'ZM Zambia',
       'ZW Zimbabwe'],
      dtype='object', length=237)

Index(['0', '00', '01', '02', '03', '04', '05', '06', '07', '08',
       ...
       '897C', '89I', '89K', '89OC', '8O', '8OC', '8OI', '8OK', '9', 'T'],
      dtype='object', length=125)

Index(['AD', 'AE', 'AF', 'AG', 'AI', 'AL', 'AM', 'AO', 'AQ', 'AR',
       ...
       'VN', 'VU', 'W1', 'WF', 'WS', 'XK', 'YE', 'ZA', 'ZM', 'ZW'],
      dtype='object', length=237)

In [4]:
from pathlib import Path
import numpy as np
out = Path('out')
out.mkdir(exist_ok=True)
slice_size = 100000

for i in np.arange(len(observations) // slice_size):
    dest_file = out / f'observations_{i:04}.csv'
    observations.iloc[i * slice_size : i * slice_size + slice_size - 1].to_csv(dest_file, index=False)

Fix up title and description as we're combining the data into one Data Cube dataset

In [5]:
from gssutils.metadata import THEME
scraper.dataset.family = 'Trade'
scraper.dataset.theme = THEME['business-industry-trade-energy']
scraper.dataset.title = scraper.dataset.title.replace('imports', 'imports and exports')
scraper.dataset.comment = scraper.dataset.comment.replace('import', 'import and export')

scraper.dataset

In [6]:
with open(out / 'dataset.trig', 'wb') as metadata:
     metadata.write(scraper.generate_trig())